In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
df=pd.read_csv(f'play_tennis.csv')
df.drop('day',axis=1,inplace=True)
df

,outlook,temp,humidity,wind,play
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


In [3]:
def Information_gain(col,Y ,label_ent): 
    uniq=[]
    for i in col:
        if i not in uniq:
            uniq.append(i)
    #######################     probabilities of each example of that specific column
    prob_of_uniq_exam=0
    cnt_of_uniq_exam=0
    probs=[]
    counts=[]
    for i in uniq:
        for j in range(0,len(col)):
            if (i==col[j]):
                cnt_of_uniq_exam+=1
        probs.append(cnt_of_uniq_exam/len(col))
        counts.append(cnt_of_uniq_exam)
        cnt_of_uniq_exam=0
    #############################        entropies of each example of that specific column
    entropies=[]
    yes_cnt=0
    no_cnt=0
    for i in range(0,len(uniq)):
        yes_cnt=0
        no_cnt=0
        for j in range (0,len(col)):
            if(uniq[i]==col[j]):
                if(Y[j]=='Yes'):
                    yes_cnt+=1
                elif(Y[j]=='No'):
                    no_cnt+=1
#         print(uniq[i],'Yes :',yes_cnt,' No :',no_cnt,'total',counts[i],probs[i])
        if(no_cnt==0):
            entropy_of_uniq_exam=((yes_cnt/counts[i])*math.log2(yes_cnt/counts[i]))
            entropy_of_uniq_exam=entropy_of_uniq_exam*-1
            entropies.append(entropy_of_uniq_exam)
        
        elif(yes_cnt==0):
            entropy_of_uniq_exam=((no_cnt/counts[i])*math.log2(no_cnt/counts[i]))
            entropy_of_uniq_exam=entropy_of_uniq_exam*-1
            entropies.append(entropy_of_uniq_exam)
            
        else:
            
            entropy_of_uniq_exam=(((yes_cnt/counts[i])*math.log2(yes_cnt/counts[i]))+((no_cnt/counts[i])*math.log2(no_cnt/counts[i])))
            entropy_of_uniq_exam=entropy_of_uniq_exam*-1
            entropies.append(entropy_of_uniq_exam)
            
#     print(entropies)
    ############################################  Information gain calculation
    temporary=0
    for i in range(0,len(entropies)):
            temporary=temporary+(probs[i]*entropies[i])
    Information_gain=label_ent-temporary
    return (Information_gain)      
    

In [4]:
def pred_class_ent(Y):
    ###############################   Appending unique classs labels
    unique_class=[]
    for i in Y:
        if not i in unique_class:
            unique_class.append(i)
    ##################################    calculating class label Entropy
    lab_ent= 0
    cnt=0
    for i in unique_class:
        cnt=0
        for j in Y:
            if (i==j):
                cnt+=1
        lab_ent=lab_ent+((cnt/len(Y))*(math.log2(cnt/len(Y))))
    lab_ent=lab_ent*-1
    return lab_ent

In [5]:
def Features_Infogain(X,Y,lab_ent):
    #######################################       #### calculating information gain   ########
    inf_gain=[]
    
    # Get the list of all column names from headers
    column_list = list(X.columns.values)
    j=0
    for i in column_list:
        inf_gain.append(Information_gain(X[i],Y ,lab_ent))
        print('Information gain of ',i,' :',inf_gain[j])
        j+=1
    return (inf_gain)

    

In [6]:
label=df.iloc[: , -1]     #  Selecting last column
featu=df.iloc[: , :-1]    # Dropping last column

In [7]:
ent=pred_class_ent(label)

In [8]:
g=Features_Infogain(featu,label,ent)

Information gain of  outlook  : 0.24674981977443933
Information gain of  temp  : 0.02922256565895487
Information gain of  humidity  : 0.15183550136234159
Information gain of  wind  : 0.04812703040826949


### Root Node

In [9]:
column_list = list(df.columns.values)
max_value = max(g)         # will return maximum probability value
ind=g.index(max_value)
print ('root node will be ',column_list[ind],' feature ')

root node will be  outlook  feature 


In [10]:
def columns_uniq(data,x):      # data= dataframe , x= column name
    tmp=[]
    for i in data[x].unique():
        tmp.append(data[data[x]==i])
    return(tmp)

In [11]:
F=columns_uniq(df,column_list[ind])
sunny=F[0]
overcast=F[1]
rain=F[2]

In [12]:
sunny=sunny.reset_index(drop=True)
overcast=overcast.reset_index(drop=True)
rain=rain.reset_index(drop=True)
sunny

,outlook,temp,humidity,wind,play
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Sunny,Mild,High,Weak,No
3,Sunny,Cool,Normal,Weak,Yes
4,Sunny,Mild,Normal,Strong,Yes


In [13]:
pl=sunny.iloc[: , -1]     #  Selecting last column
ent1=pred_class_ent(pl)
sunny.drop('outlook',inplace=True, axis=1)   # Will drop 1st column
sunny=sunny.iloc[: , :-1]    # Dropping last column   # Will drop last column
if(ent1==0):
    print(ent1)
else:
    sun=Features_Infogain(sunny,pl,ent1)
    
#outlook==sunny, humidity==Normal  --------------------->play=Yes

Information gain of  temp  : 0.5709505944546686
Information gain of  humidity  : 0.9709505944546686
Information gain of  wind  : 0.01997309402197489


In [14]:
overcast

,outlook,temp,humidity,wind,play
0,Overcast,Hot,High,Weak,Yes
1,Overcast,Cool,Normal,Strong,Yes
2,Overcast,Mild,High,Strong,Yes
3,Overcast,Hot,Normal,Weak,Yes


In [15]:
pl=overcast.iloc[: , -1]     #  Selecting last column
ent2=pred_class_ent(pl)
overcast.drop('outlook',inplace=True, axis=1)   # Will drop 1st column
overcast=overcast.iloc[: , :-1]    # Dropping last column   # Will drop last column
if(ent2==0):
    print(ent2)
else:
    sun=Features_Infogain(overcast,pl,ent2)
# outlook==Overcast  --------------------->play=Yes

-0.0


In [16]:
rain

,outlook,temp,humidity,wind,play
0,Rain,Mild,High,Weak,Yes
1,Rain,Cool,Normal,Weak,Yes
2,Rain,Cool,Normal,Strong,No
3,Rain,Mild,Normal,Weak,Yes
4,Rain,Mild,High,Strong,No


In [17]:
pl=rain.iloc[: , -1]     #  Selecting last column
ent3=pred_class_ent(pl)
rain.drop('outlook',inplace=True, axis=1)   # Will drop 1st column
rain=rain.iloc[: , :-1]    # Dropping last column   # Will drop last column
if(ent3==0):
    print(ent3)
else:
    sun=Features_Infogain(rain,pl,ent3)
#outlook==sunny, wind==Weak  --------------------->play=Yes

Information gain of  temp  : 0.01997309402197489
Information gain of  humidity  : 0.01997309402197489
Information gain of  wind  : 0.9709505944546686


In [18]:
def predict(array):
    
    if array[0]=='Overcast':
        return 'Yes'
    elif array[0]=='Sunny':
        
        if array[2]=='High':
            return 'No'
        elif array[2]=='Normal':
            return 'Yes'
    elif array[0]=='Rain':
        if array[3]=='Weak':
            return 'Yes'
        elif array[3]=='Strong':
            return 'No'

In [19]:
list1= ['Rain' , 'Cool' , 'High' , 'Weak']
predict(list1)

'Yes'